In [13]:
import scipy.io
import numpy as np


In [27]:
#Load the .mat file
data = scipy.io.loadmat('./SEED/SEED_EEG/Preprocessed_EEG/1_20131027.mat')

#Extrct EEg data and labels
eeg_segments = [data[f'djc_eeg{i}'] for i in range(1, 16)]
# labels = data['labels'].flatten()

#convert to numpy arrays for processing 
# eeg_data = np.stack(eeg_segments, axis=0)
print(data.keys())
print(eeg_segments)
for i, eeg in enumerate(eeg_segments, start=1):
    print(f"djc_eeg{i} shape: {eeg.shape}")

# print(labels)
# print(f"EEG Data shape: {eeg_data.shape}")
# print(f"Labels Shape: {labels.shape}")

dict_keys(['__header__', '__version__', '__globals__', 'djc_eeg1', 'djc_eeg2', 'djc_eeg3', 'djc_eeg4', 'djc_eeg5', 'djc_eeg6', 'djc_eeg7', 'djc_eeg8', 'djc_eeg9', 'djc_eeg10', 'djc_eeg11', 'djc_eeg12', 'djc_eeg13', 'djc_eeg14', 'djc_eeg15'])
[array([[-11.62290573, -16.21246338, -17.13633537, ..., -46.10419273,
        -51.05137825, -43.6604023 ],
       [ 14.93096352,   8.07642937,   8.19563866, ..., -24.31869507,
        -27.74596214, -22.5007534 ],
       [ 38.05756569,  25.57039261,  37.99796104, ..., -54.06141281,
        -58.29334259, -48.63739014],
       ...,
       [ 24.25909042,  15.49720764,  21.51727676, ...,   7.62939453,
         17.34495163,  -8.5234642 ],
       [ 26.82209015,  27.77576447,  26.70288086, ...,  25.74920654,
         35.76278687,  16.00384712],
       [ 29.77252007,  33.55741501,  30.39836884, ...,  20.02716064,
         30.15995026,  12.15934753]]), array([[-57.6376915 , -41.15700722, -41.45503044, ..., -16.65949821,
        -28.58042717, -35.04753113],
 

In [36]:
import os

folder_path = './SEED/SEED_EEG/Preprocessed_EEG'

# Initialize variables
all_files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]
smallest_length = float('inf')

# Step 1: Find the smallest trial length
for file_name in all_files:
    file_path = os.path.join(folder_path, file_name)
    data = scipy.io.loadmat(file_path)
    
    # Iterate through trials in the file
    for i in range(1, 16):
        key = f'djc_eeg{i}'
        if key in data:
            trial = data[key]
            smallest_length = min(smallest_length, trial.shape[1])
        
print(f"Smallest trial length: {smallest_length}")

Smallest trial length: 37001


In [41]:
from scipy.interpolate import interp1d

# Function to resample a trial to a fixed length
def resample_trial(trial, target_length):
    channels, time_points = trial.shape
    resampled = np.zeros((channels, target_length))
    for ch in range(channels):
        interp = interp1d(np.linspace(0,1,time_points), trial[ch,:], kind='linear') 
        resampled[ch,:] = interp(np.linspace(0,1,target_length))
    return resampled

In [ ]:
def normalize_eeg(data):
    mean = np.mean(data, axis=(0,2), keepdims=True)
    std = np.std(data, axis=(0,2), keepdims=True)
    return (data - mean) / std

In [43]:
# Initialize lists for resampled and normalized data
all_data = []
all_labels = []

# Resample and normalize all trials
for file_name in all_files:
    file_path = os.path.join(folder_path, file_name)
    data = scipy.io.loadmat(file_path)
    
    # Extract EEG segments and labels
    eeg_segments = [data[f'djc_eeg{i}'] for i in range(1, 16) if f'djc_eeg{i}' in data]
    labels = np.array([1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1])  # Provided labels
    
    # Resample and normalize each trial
    resample_trials = [resample_trial(trial, smallest_length) for trial in eeg_segments]
    normalized_trials = [normalize_eeg(trial) for trial in resample_trials]
    
    # Add to the dataset
    all_data.extend(normalized_trials)
    all_labels.extend(labels[:len(normalized_trials)])
    
# Convert to numpy arrays
all_data = np.stack(all_data, axis=0)
all_labels = np.array(all_labels)

print(f"Final Data shape: {all_data.shape}")
print(f"Final Labels shape: {all_labels.shape}")



Final Data shape: (45, 62, 37001)
Final Labels shape: (45,)
